### **Testing various NN's on *multivariate* time series**

##### Imports

In [ ]:
# N-BEATS
from neuralforecast.models import NBEATS, PatchTST, NBEATSx
from neuralforecast.losses.pytorch import HuberLoss
from neuralforecast.core import NeuralForecast

import joblib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, make_scorer
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import RandomizedSearchCV

# Hyperparameter tuning
import optuna

# N-HiTS
from darts import TimeSeries
from darts.models import NHiTSModel
from torch.nn import MSELoss


##### Standard scaling

In [ ]:
def standard_scaling(x):
    mean = np.mean(np.abs(x))
    s = np.std(x)
    if s == 0:
        return x    
    return (x - mean)/s

def standard_unscaling(original, scaled):
    mean = np.mean(np.abs(original))
    s = np.std(original)

    return (scaled * s) + mean

### **1** N-BEATSx

#### **1.1** Ground water data

In [ ]:
# Get the data
aquifer_by_stations = joblib.load('../data/interim/ground-water-and-weather-no-new-features.joblib')

In [ ]:
# Function to add the predictions
def weather_forecast(X):
    postfixes = ['avg', 'min', 'max']
    for postfix in postfixes:
        X[f"precipitation_probability_{postfix}_shift-{5}"] = X[f'precipitation_probability_{postfix}'].shift(-5)
        X[f"precipitation_intensity_{postfix}_shift-{5}"] = X[f'precipitation_intensity_{postfix}'].shift(-5)
    return X

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Add the weather forecast features
for aquifer in aquifers_list:
    aquifer_by_stations[aquifer] = weather_forecast(aquifer_by_stations[aquifer])
    aquifer_by_stations[aquifer] = aquifer_by_stations[aquifer][:-5]

In [ ]:
# The additional parameters that will be used for training and prediction
additional_parameters = aquifer_by_stations[85065].columns.values.tolist()

# Clean up the list
parameters_to_delete = ['date', 'station_id', 'altitude', 'altitude_diff', 'id', 'location_id']
additional_parameters = [parameter for parameter in additional_parameters if parameter not in parameters_to_delete]

##### Hyperparameter tuning

In [ ]:
# Define the horizon and the day_len
horizon = 5
day_len = 100
test_len = 365

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Scale the additional features
for aquifer in aquifers_list:
    for feature in additional_parameters:
        aquifer_by_stations[aquifer][feature] = standard_scaling(aquifer_by_stations[aquifer][feature])

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_size = trial.suggest_int('input_size', 5, 20)
    
    n_harmonics = trial.suggest_int('n_harmonics', 1, 5)
    n_polynomials = trial.suggest_int('n_polynomials', 1, 5)
    
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1)

    max_steps = trial.suggest_int('max_steps', 10, 600)

    validation_size = trial.suggest_int('val_size', 5, 15)

    scaling = trial.suggest_categorical('scaling', [True, False])

    

    models = [NBEATSx(h=horizon,input_size=input_size,
                 max_steps=max_steps,
                 learning_rate=learning_rate,
                 n_harmonics=n_harmonics,
                 n_polynomials=n_polynomials,
                 hist_exog_list=additional_parameters,
                 accelerator='cuda',
                 logger=False)
                 ]
    model = NeuralForecast(models=models, freq='D')

    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for aquifer in aquifers_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = aquifer_by_stations[aquifer][:-test_len]

        # Rename the columns (library wants to have specific names)
        y = y.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})

        # Scaling
        if scaling:
            y['y'] = standard_scaling(y['y'])

        # Fit the model
        model.fit(y[:-day_len], val_size=validation_size)

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(df=y[:-i], verbose=0)

            # Unscale
            if scaling:
                forecast['NBEATSx'] = standard_unscaling(aquifer_by_stations[aquifer]['altitude_diff'], forecast['NBEATSx'])
                

            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast['NBEATSx'].values[i])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(y['y'][-day_len:], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    print(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Testing on multiple stations

In [ ]:
aquifers_list = [85065, 85064]

<span style="color:red"><sup>!!! The scaling is mandatory, otherwise the results are very bad</sup></span>

In [ ]:
# Scale the additional features
for aquifer in aquifers_list:
    for feature in additional_parameters:
        aquifer_by_stations[aquifer][feature] = standard_scaling(aquifer_by_stations[aquifer][feature])

In [ ]:
'''{'input_size': 7,
 'n_harmonics': 2,
 'n_polynomials': 3,
 'learning_rate': 0.009866118273076567,
 'max_steps': 520,
 'val_size': 9,
 'scaling': True}'''

In [ ]:
horizon = 5 # prediction horizon
day_len = 365 # number of days to forecast
val_size = 9

models = [NBEATSx(h=horizon, 
                 accelerator='cuda',
                 input_size=15,
                 #n_harmonics=2,
                 #n_polynomials=3,
                 #learning_rate=0.01,
                 max_steps=500,
                 hist_exog_list=additional_parameters,
                 devices=[0],
                 logger=False)]

model = NeuralForecast(models=models, freq='D')

# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for aquifer in aquifers_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = aquifer_by_stations[aquifer]

    # Rename the columns (library wants to have specific names)
    y = y.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})

    # Scaling
    y['y'] = standard_scaling(y['y'])

    # Only keep these 3 columns
    #y = y[['ds', 'y', 'unique_id']]

    # Fit the model
    model.fit(y[:-day_len], val_size=val_size)

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Predict
        forecast = model.predict(df=y[:-i], verbose=0)

        # Unscale
        forecast['NBEATSx'] = standard_unscaling(aquifer_by_stations[aquifer]['altitude_diff'], forecast['NBEATSx'])

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast['NBEATSx'].values[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-day_len:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[aquifer]['date'][-350:-250], aquifer_by_stations[aquifer]['altitude_diff'][-350:-250], color="royalblue", label="true data")
#plt.plot(aquifer_by_stations[aquifer]['date'][-200:], aquifer_by_stations[aquifer]['precipitation'][-200:].apply(lambda x: x/50), color="olive", label="true data")
#plt.plot(aquifer_by_stations[aquifer]['date'][-200:], predictions[0][-200:], color="tomato", label="forecast")
#plt.plot(aquifer_by_stations[aquifer]['date'][-200:], predictions[1][-200:], color="green", label="forecast")
plt.plot(aquifer_by_stations[aquifer]['date'][-350:-250], predictions[4][-350:-250], color="goldenrod", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-beatsx/n-beatxs-ground-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-beatsx/n-beatsx-ground-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(aquifers_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-beatsx/n-beatsx-ground-water-r2-stations.joblib')

#### **1.2** Surface water data

In [ ]:
# Read the dataset
watercourse_by_stations = joblib.load('../data/interim/surface-water-and-weather-no-new-features.joblib')

In [ ]:
# List of station used for testing
station_list = [2530, 2620, 4200, 4230, 4270, 4515, 4520, 4570, 4575, 5040, 5078, 5330, 5425, 5500, 6060, 6068, 6200, 6220, 6300, 6340, 8454, 8565]

In [ ]:
# Add the weather forecast features
for station in station_list:
    watercourse_by_stations[station] = weather_forecast(watercourse_by_stations[station])
    watercourse_by_stations[station] = watercourse_by_stations[station][:-5]

In [ ]:
# The additional parameters that will be used for training and prediction
additional_parameters = watercourse_by_stations[2530].columns.values.tolist()

# Clean up the list
parameters_to_delete = ['date', 'station_id', 'level', 'level_diff', 'id', 'location_id']
additional_parameters = [parameter for parameter in additional_parameters if parameter not in parameters_to_delete]

##### Hyperparemeter tuning

In [ ]:
# Define the horizon and the day_len
horizon = 5
day_len = 100
test_len = 365

In [ ]:
station_list = [4270, 4570, 4515, 6068]

In [ ]:
# Scale the additional features
for station in station_list:
    for feature in additional_parameters:
        watercourse_by_stations[station][feature] = standard_scaling(watercourse_by_stations[station][feature])

In [ ]:
# Define the function which contains parameters to tune the model

def objective(trial):
    input_size = trial.suggest_int('input_size', 5, 20)
    
    n_harmonics = trial.suggest_int('n_harmonics', 1, 5)
    n_polynomials = trial.suggest_int('n_polynomials', 1, 5)
    
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1)

    max_steps = trial.suggest_int('max_steps', 10, 1000)

    validation_size = trial.suggest_int('val_size', 5, 15)

    models = [NBEATSx(h=horizon,input_size=input_size,
                 max_steps=max_steps,
                 learning_rate=learning_rate,
                 n_harmonics=n_harmonics,
                 n_polynomials=n_polynomials,
                 hist_exog_list=additional_parameters,
                 accelerator='cuda',
                 logger=False)
                 ]
    model = NeuralForecast(models=models, freq='D')

    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for station in station_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = watercourse_by_stations[station][:-test_len]

        # Rename the columns (library wants to have specific names)
        y = y.rename(columns={'date':'ds', 'level_diff':'y', 'station_id':'unique_id'})

        # Fit the model
        model.fit(y[:-day_len], val_size=validation_size)

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(df=y[:-i], verbose=0)

            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast['NBEATS'].values[i])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(y['y'][-day_len:], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    print(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Testing on multiple stations

In [ ]:
# List of station used for testing
station_list = [2530, 2620, 4200, 4230, 4270, 4515, 4520, 4570, 4575, 5040, 5078, 5330, 5425, 5500, 6060, 6068, 6200, 6220, 6300, 6340, 8454, 8565]

In [ ]:
station_list = [2530]

In [ ]:
# Scale the additional features
for station in station_list:
    for feature in additional_parameters:
        watercourse_by_stations[station][feature] = standard_scaling(watercourse_by_stations[station][feature])

In [ ]:
horizon = 5 # prediction horizon
day_len = 365 # number of days to forecast
val_size = 2*horizon

models = [NBEATSx(h=horizon, 
                 #loss=HuberLoss(),
                 accelerator='cuda',
                 input_size=2*horizon,
                 #n_harmonics=2,
                 #n_polynomials=4,
                 #scaler_type='robust',
                 #learning_rate=0.01,
                 max_steps=500,
                 #n_blocks=[1, 1, 3],
                 #mlp_units=[[32, 32], [32, 32]],
                 hist_exog_list=additional_parameters,
                 logger=False)]

model = NeuralForecast(models=models, freq='D')

# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for station in station_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = watercourse_by_stations[station]

    # Rename the columns (library wants to have specific names)
    y = y.rename(columns={'date':'ds', 'level_diff':'y', 'station_id':'unique_id'})

    # Scale the level_diff
    #y['y'] = standard_scaling(y['y'])

    # Fit the model
    model.fit(y[:-day_len], val_size=horizon)

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Predict
        forecast = model.predict(df=y[:-i], verbose=0)

        # Unscale the predictions
        #forecast['NBEATSx'] = standard_unscaling(watercourse_by_stations[station]['level_diff'], forecast['NBEATSx'])

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast['NBEATSx'].values[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-day_len:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(watercourse_by_stations[station]['level_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(watercourse_by_stations[station]['date'][-200:], watercourse_by_stations[station]['level_diff'][-200:], color="royalblue", label="true data")
plt.plot(watercourse_by_stations[station]['date'][-200:], predictions[0][-200:], color="tomato", label="forecast")
plt.plot(watercourse_by_stations[station]['date'][-200:], predictions[4][-200:], color="goldenrod", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-beatsx/n-beatsx-surface-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-beatsx/n-beatsx-surface-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(station_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-beatsx/n-beatsx-surface-water-r2-stations.joblib')

### **2** N-HiTS

#### **2.1** Ground water data

In [ ]:
# Get the data
aquifer_by_stations = joblib.load('../data/interim/ground-water-and-weather-no-new-features.joblib')

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Add the weather forecast features
for aquifer in aquifers_list:
    aquifer_by_stations[aquifer] = weather_forecast(aquifer_by_stations[aquifer])
    aquifer_by_stations[aquifer] = aquifer_by_stations[aquifer][:-5]

In [ ]:
# The additional parameters that will be used for training and prediction
additional_parameters = aquifer_by_stations[85065].columns.values.tolist()

# Clean up the list
parameters_to_delete = ['date', 'station_id', 'altitude', 'altitude_diff', 'id', 'location_id']
additional_parameters = [parameter for parameter in additional_parameters if parameter not in parameters_to_delete]

##### Hyperparameter tuning

In [ ]:
# Define the horizon, day_len (number of predicted days), test_len (number of days used for final testing)
horizon = 5
day_len = 100
test_len = 365

In [ ]:
# Stations to test
aquifers_list = [85065, 85064]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_chunk_length = trial.suggest_int('input_chunk_length', 5, 70)
    output_chunk_length = trial.suggest_int('output_chunk_length', 1, 10)
    num_stacks = trial.suggest_int('num_stacks', 1, 4)
    num_blocks = trial.suggest_int('num_blocks', 1, 3)
    num_layers = trial.suggest_int('num_layers', 2, 5)
    layer_widths = trial.suggest_categorical('layer_widths', [64, 128, 256, 512])
    dropout = trial.suggest_categorical('dropout', [0.1, 0.2])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-2, 1e-3, 1e-4])
    n_epochs = trial.suggest_int('n_epochs', 10, 200)

    model = NHiTSModel(input_chunk_length=input_chunk_length,
                     output_chunk_length=output_chunk_length,
                     num_stacks=num_stacks,
                     num_blocks=num_blocks,
                     num_layers=num_layers,
                     layer_widths=layer_widths,
                     dropout=dropout,
                     optimizer_kwargs={'lr': learning_rate},
                     n_epochs=n_epochs,
                     pl_trainer_kwargs={'logger': False, "accelerator": "gpu", "devices": [0]})
    


    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for aquifer in aquifers_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = aquifer_by_stations[aquifer][:-test_len]

        # Change to TimeSeries format (required by the library)
        y = TimeSeries.from_dataframe(y, time_col='date', value_cols='altitude_diff')

        # Fit the model
        model.fit(y[:-day_len])

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(n=horizon, series=y[:-i])


            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast.values()[i][0])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-(day_len+test_len):-test_len], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    print(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Test on multiple stations

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Scale the additional features
for aquifer in aquifers_list:
    for feature in additional_parameters:
        aquifer_by_stations[aquifer][feature] = standard_scaling(aquifer_by_stations[aquifer][feature])

In [ ]:
aquifers_list = [85065]

In [ ]:
'''{'input_chunk_length': 46,
 'output_chunk_length': 8,
 'num_stacks': 3,
 'num_blocks': 2,
 'num_layers': 4,
 'layer_widths': 512,
 'dropout': 0.2,
 'learning_rate': 0.0001,
 'n_epochs': 89}'''

In [ ]:
horizon = 5 # prediction horizon
day_len = 365 # number of days to forecast

# Set the model parameters
model = NHiTSModel(
    input_chunk_length=46,
    output_chunk_length=8,
    #num_blocks=2,
    #num_stacks=3,
    #num_layers=4,
    #layer_widths=512,
    #dropout=0.2,
    n_epochs=20,
    optimizer_kwargs={'lr': 1e-4},
    pl_trainer_kwargs={'logger': False, "accelerator": "gpu", "devices": -1}
)


# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for aquifer in aquifers_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = aquifer_by_stations[aquifer].copy()

    # Scale the target values
    #y['altitude_diff'] = standard_scaling(y['altitude_diff'])

    # Change the format to TimeSeries
    y = TimeSeries.from_dataframe(y, time_col='date', value_cols='altitude_diff')
    
    # Change the additional features to TimeSeries
    additional_parameters_TS  = TimeSeries.from_dataframe(aquifer_by_stations[aquifer], time_col='date', 
                                                          value_cols=additional_parameters)

    # Fit the model
    model.fit(y[:-day_len], past_covariates=additional_parameters_TS[:-day_len])

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Make predictions
        forecast = model.predict(n=horizon, series=y[:-i], past_covariates=additional_parameters_TS[:-i])

        # Unscale the predictions
        #for i in range(horizon):
            #forecast.values()[i][0] = standard_unscaling(aquifer_by_stations[aquifer]['altitude_diff'], forecast.values()[i][0])

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast.values()[i][0])
    
    # Clean up the results
    predictions[0] = predictions[0][-day_len:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[aquifer]['date'][-200:], aquifer_by_stations[aquifer]['altitude_diff'][-200:], color="royalblue", label="true data")
plt.plot(aquifer_by_stations[aquifer]['date'][-200:], predictions[0][-200:], color="tomato", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-hits/n-hits-ground-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-hits/n-hits-ground-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(aquifers_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-hits/n-hits-ground-water-r2-stations.joblib')